In [1]:
import requests
import json
import pandas as pd
import plotly.graph_objects as go
import datetime
import dateutil.parser

#if plotly doesn't work:
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot
init_notebook_mode(connected=True)

print('Notebook initialized.')

Notebook initialized.


In [25]:
url = "https://api.applicationinsights.io/v1/apps/b584432b-34cb-4369-82fd-026dbf13d49f/query"
startDate = '2020-02-01'
endDate = '2020-04-01'
dateQuery = f"| where timestamp > datetime('{startDate}') and timestamp < datetime('{endDate}')"
params = {
    "query":
    f"""
let InteractionData = customEvents
{dateQuery}
| where client_Browser contains '' or '' == ''
| where name == 'Register.Web.Interaction'
| where 'false' == 'true'  or (customDimensions.userName != 'Win&*^!£$#&Tech'
and customDimensions.userName !startswith 'rt_'
and customDimensions.userName != 'ThisIsAReallyLonggggggggggggUsernameThisIsAReallyLonggggggggggggUsername'
and customDimensions.userName != 'bw_valid_user'
and customDimensions.userName != 'e'
and (customDimensions.userName != '' or isnull(customDimensions.userName) == true) 
and customDimensions.userName != 'R')
| extend territ = tostring(customDimensions.territory)
| summarize timestamp_max = max(timestamp) by session_Id, name, territ;
let SubmitData = InteractionData
| join kind= inner
(
    customEvents
    {dateQuery}
    | where name == 'Register.Web.SubmitForm'
    | extend name = 'Submit'
)
on session_Id
| summarize timestamp_max1 = max(timestamp) by name1, session_Id, territ;
let ValidSubmitData = InteractionData
| join kind= inner
(
    customEvents
    {dateQuery}
    | where name == 'Register.Web.SubmitFormValid' or  name == 'ServiceController.PostResult()'
    | extend name = 'validSubmit'
)
on session_Id
| summarize timestamp_max1 = max(timestamp) by name1, session_Id, territ;
let SuccessData = InteractionData
| join kind= inner
(
    customEvents
    {dateQuery}
    | where name == 'Register.Web.Successful' or  name == 'Register.Web.Server.Successful'
    | extend name = 'successReg'
)
on session_Id
| summarize timestamp_max1 = max(timestamp) by name1, session_Id, territ;
let FailsData = ValidSubmitData
| join kind= leftouter
(
    SuccessData
)
on session_Id
| where name11 ==''
| extend name1 = 'Fails';
InteractionData
| extend name1 = name, timestamp_max1 = timestamp_max
| project name1, session_Id, timestamp_max1, territ
| union  SuccessData
| union  ValidSubmitData
| union  FailsData
| extend _name = case(
name1 == 'Register.Web.Interaction', 'Interact', 
name1 == 'validSubmit', 'ValidSub', 
name1 == 'successReg', 'SuccessReg',
name1 == 'Fails', 'FailedReg',
'error')
| summarize count() by _name, territ
| where territ != 'Unknown'
| project territ, count_, _name
| render timechart
"""
         }


params_withdates = {
    "query":
    f"""
let InteractionData = customEvents
{dateQuery}
| where client_Browser contains '' or '' == ''
| where name == 'Register.Web.Interaction'
| where 'false' == 'true'  or (customDimensions.userName != 'Win&*^!£$#&Tech'
and customDimensions.userName !startswith 'rt_'
and customDimensions.userName != 'ThisIsAReallyLonggggggggggggUsernameThisIsAReallyLonggggggggggggUsername'
and customDimensions.userName != 'bw_valid_user'
and customDimensions.userName != 'e'
and (customDimensions.userName != '' or isnull(customDimensions.userName) == true) 
and customDimensions.userName != 'R')
| extend territ = tostring(customDimensions.territory)
| summarize timestamp_max = max(timestamp) by session_Id, name, territ;
let SubmitData = InteractionData
| join kind= inner
(
    customEvents
    {dateQuery}
    | where name == 'Register.Web.SubmitForm'
    | extend name = 'Submit'
)
on session_Id
| summarize timestamp_max1 = max(timestamp) by name1, session_Id, territ;
let ValidSubmitData = InteractionData
| join kind= inner
(
    customEvents
    {dateQuery}
    | where name == 'Register.Web.SubmitFormValid' or  name == 'ServiceController.PostResult()'
    | extend name = 'validSubmit'
)
on session_Id
| summarize timestamp_max1 = max(timestamp) by name1, session_Id, territ;
let SuccessData = InteractionData
| join kind= inner
(
    customEvents
    {dateQuery}
    | where name == 'Register.Web.Successful' or  name == 'Register.Web.Server.Successful'
    | extend name = 'successReg'
)
on session_Id
| summarize timestamp_max1 = max(timestamp) by name1, session_Id, territ;
let FailsData = ValidSubmitData
| join kind= leftouter
(
    SuccessData
)
on session_Id
| where name11 ==''
| extend name1 = 'Fails';
InteractionData
| extend name1 = name, timestamp_max1 = timestamp_max
| project name1, session_Id, timestamp_max1, territ
| union  SuccessData
| union  ValidSubmitData
| union  FailsData
| extend _name = case(
name1 == 'Register.Web.Interaction', 'Interact', 
name1 == 'validSubmit', 'ValidSub', 
name1 == 'successReg', 'SuccessReg',
name1 == 'Fails', 'FailedReg',
'error')
| summarize count() by _name, territ, bin(timestamp_max1, 1d)
| where territ != 'Unknown'
| project territ, count_, _name, timestamp = timestamp_max1
| render timechart
    """
}

headers = {"x-api-key":"bbw2l1nrwczgsg8lbq44ywisokg0tecqa2wfzc70"}

resp = requests.post(url=url, headers=headers, json=params, verify=True)
data = resp.json()
rows = data["tables"][0]["rows"]

df = pd.DataFrame(rows) 
df.columns = ['territory', 'count', 'name']
df = df.sort_values(by=['count'], ascending=False)
df_inte = df.query('name == "Interact"')
df_vsub = df.query('name == "ValidSub"')
df_succ = df.query('name == "SuccessReg"')
df_fails = df.query('name == "FailedReg"')

resp2 = requests.post(url=url, headers=headers, json=params_withdates, verify=True)
data2 = resp2.json()

rows2 = data2["tables"][0]["rows"]
df2 = pd.DataFrame(rows2)
df2.columns = ['territory', 'count', 'name', 'timestamp']
df2 = df2.sort_values(by=['timestamp'])
df2_succ = df2.query('name == "SuccessReg"')


names = ['Interact', 'ValidSub', 'SuccessReg', 'Fails']
goodnames = ['Interactions', 'Valid Submits', 'Successful Registrations', 'Failed Registrations']
dfs = [df_inte, df_vsub, df_succ, df_fails]
colors = ['green', 'orange', 'blue', 'red']

df_countries = ["GBR", "IRL", "ITA", "DEU", "ESP"]
df_short = df[df['territory'].isin(df_countries)].sort_values(by=['territory'])
df2_short = df2[df2['territory'].isin(df_countries)].sort_values(by=['territory'])
df_short_succ = df_short.query('name == "SuccessReg"')
df2_short_succ = df2_short.query('name == "SuccessReg"')
df2_final = df2_short_succ.assign(event="")

print(df_succ.head(5))
print(df2_succ.head(5))

    territory  count        name
165       GBR  57075  SuccessReg
164       IND  50984  SuccessReg
168       DEU  18831  SuccessReg
163       MEX  17809  SuccessReg
176       CAN  16797  SuccessReg
     territory  count        name             timestamp
7533       MLI      4  SuccessReg  2020-02-01T00:00:00Z
8649       SLE      1  SuccessReg  2020-02-01T00:00:00Z
8648       IDN      1  SuccessReg  2020-02-01T00:00:00Z
8647       CZE      1  SuccessReg  2020-02-01T00:00:00Z
8646       TGO      1  SuccessReg  2020-02-01T00:00:00Z


In [33]:
def prepareGraph(df, countryName, color):
    fig = go.Figure()
    df_act = df.query('territory == "' + countryName +'"').sort_values(by=['timestamp'])
    fig.add_trace(
        go.Scatter(
            name=countryName,
            x = df_act['timestamp'],
            y = df_act['count'],
            mode="lines",
            text=df_act['event'],
            textposition="top center", 
            line_color=color
        )
    )
    return fig

def CreateAnnot(array, x, y, text, ay):
    di = dict(
            x=x,
            y=y,
            xref="x",
            yref="y",
            text=text,
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=ay
        )
    array.append(di)
    return array

def AddAnnot(fig, annotArray, country):
    fig.update_layout(
        xaxis_title="Time",
        yaxis_title="Count",
        title=country,
        annotations=annotArray
    )
    return fig

print("Functions ready")

Functions ready


In [45]:
figGBR = prepareGraph(df2_final, "GBR", "blue")
annotArray = CreateAnnot([], "2020-03-10T00:00:00Z", 4795, "Cheltenham festival", -30)
annotArray = CreateAnnot(annotArray, "2020-03-20T00:00:00Z", 376, "School closures", -40)
annotArray = CreateAnnot(annotArray, "2020-03-23T00:00:00Z", 379, "Stay at home announcement", -100)
figGBR = AddAnnot(figGBR, annotArray, "Daily registrations - UK")
figGBR.show()

figDEU = prepareGraph(df2_final, "DEU", "orange")
annotArray = CreateAnnot([], "2020-03-13T00:00:00Z", 218, "School closures", -150)
annotArray = CreateAnnot(annotArray, "2020-03-22T00:00:00Z", 280, "National Lockdown", -150)
figDEU = AddAnnot(figDEU, annotArray, "Daily registrations - Germany")
figDEU.show()

figITA = prepareGraph(df2_final, "ITA", "green")
annotArray = CreateAnnot([], "2020-02-21T00:00:00Z", 95, "Lombardy Lockdown", -150)
annotArray = CreateAnnot(annotArray, "2020-03-04T00:00:00Z", 62, "School closures", -150)
annotArray = CreateAnnot(annotArray, "2020-03-08T00:00:00Z", 100, "Northern Italy Lockdown", -150)
annotArray = CreateAnnot(annotArray, "2020-03-09T00:00:00Z", 72, "National Lockdown", 30)
figITA = AddAnnot(figITA, annotArray, "Daily registrations - Italy")
figITA.show()

figESP = prepareGraph(df2_final, "ESP", "red")
annotArray = CreateAnnot([], "2020-03-12T00:00:00Z", 94, "School closures", 30)
annotArray = CreateAnnot(annotArray, "2020-03-23T00:00:00Z", 105, "National Lockdown", -150)
figESP = AddAnnot(figESP, annotArray, "Daily registrations - Spain")
figESP.show()

figIRL = prepareGraph(df2_final, "IRL", "black")
annotArray = CreateAnnot([], "2020-03-10T00:00:00Z", 402, "Cheltenham festival", -140)
annotArray = CreateAnnot(annotArray, "2020-03-12T00:00:00Z", 244, "School closures", -150)
annotArray = CreateAnnot(annotArray, "2020-03-27T00:00:00Z", 25, "National Lockdown", -150)
figIRL = AddAnnot(figIRL, annotArray, "Daily registrations - Ireland")
figIRL.show()